Вробе бы дешевый прокси  
https://www.ttproxy.com/pricing/

In [1]:
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
import numpy as np
from time import sleep
import time
from random import randint
from datetime import datetime
import pymysql.cursors
import threading
import concurrent.futures

###  Процедура по выводу progressbar

In [2]:
def progress(count, total, suffix=''):
    bar_len = 40
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 2)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()

In [3]:
START_URL = 'https://www.b2b-center.ru/market/?show=archive'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '+\
           'Chrome/86.0.4240.111 Safari/537.36',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;'+\
           'q=0.8,application/signed-exchange;v=b3;q=0.9'}
HOST = 'https://www.b2b-center.ru'
REC_PER_PAGE = 20
PROJECT_ID = 1
FALL_COUNT = 5 # Кол-во неудачных соединений (запись в базе), после которого прокси не берется в работу

In [4]:
MYSQL_HOST='localhost'
MYSQL_USER='proxy_user'
MYSQL_PWD='proxy_user12345'
MYSQL_DB='proxy_servers'

# Чтение из базы
def mysql_read(sql):
    connection = pymysql.connect(host=MYSQL_HOST,
                                 user=MYSQL_USER,
                                 password=MYSQL_PWD,
                                 db=MYSQL_DB,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            return cursor
    finally:
        connection.close()

# Запись в базу
def mysql_write(sql):
    connection = pymysql.connect(host=MYSQL_HOST,
                                 user=MYSQL_USER,
                                 password=MYSQL_PWD,
                                 db=MYSQL_DB,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(sql)
        # connection is not autocommit by default. So you must commit to save
        # your changes.
        connection.commit()

    finally:
        connection.close()

In [5]:
MYSQL_HOST2='localhost'
MYSQL_USER2='proxy_user'
MYSQL_PWD2='proxy_user12345'
MYSQL_DB2='b2b_center'

# Чтение из базы
def mysql_read2(sql, values=None):
    connection = pymysql.connect(host=MYSQL_HOST2,
                                 user=MYSQL_USER2,
                                 password=MYSQL_PWD2,
                                 db=MYSQL_DB2,
                                 charset='utf8mb4')

    try:
        with connection.cursor() as cursor:
            if values:
                cursor.execute(sql, values)
            else:
                cursor.execute(sql)
            
            return cursor
    finally:
        connection.close()

# Запись в базу
def mysql_write2(sql, values=None):
    connection = pymysql.connect(host=MYSQL_HOST2,
                                 user=MYSQL_USER2,
                                 password=MYSQL_PWD2,
                                 db=MYSQL_DB2,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            if values:
                cursor.execute(sql, values)
            else:
                cursor.execute(sql)
        # connection is not autocommit by default. So you must commit to save
        # your changes.
        connection.commit()
        return cursor

    finally:
        connection.close()

### Иницилизация DataFrame

In [22]:
def df_init():
    global df_columns
    global df
    df_columns=['Id','Type','Name','Sponsor','Link','Start_date','End_date']
    df = pd.DataFrame(columns=df_columns)
    # df['Id'] = df['Id'].astype('int64')

### Функция по заполнению ДатаФрейма данными из страницы со списком закупок

In [6]:
def parse_list_page(html):
    soup = BeautifulSoup(html.text, 'html.parser')
    search_table = soup.find_all('table', class_='table table-hover table-filled search-results')

    links = search_table[0].tbody.find_all('a')
    dates = search_table[0].tbody.find_all('td', class_='nowrap')

    for i in range(0, len(links), 2):
        auc_id = links[i].text.split('\n')[0].split('№')[1].strip() # Номер торгов
        print(auc_id)
        if not len(df[df['Id'] == auc_id]):
            auc_type = links[i].text.split('\n')[0].split('№')[0].strip() # Вид торгов
            auc_name = links[i].text.split('\n')[1].strip() # if len(links[i].text.split('\n')) > 1 else '11' # Название торгов
            auc_name = np.nan if len(auc_name) == 0 else auc_name
            sponsor = np.nan if len(links[i+1].text.strip()) == 0 else links[i+1].text.strip() # Наименование органзатора
            auc_link = HOST + links[i].get('href').split('#')[0] # Ссылка на страницу торгов
            start_date = dates[i].text # Колонка "Опубликовано"
            end_date = dates[i+1].text # Колонка "Актуально до"
            add_list = [auc_id, auc_type, auc_name, sponsor, auc_link, start_date, end_date]
            df.loc[len(df)] = add_list

In [721]:
%%time
prox = 'socks5://183.236.164.121:1081'
prox = 'https://95.216.10.19:3128'
proxies = {
  'http': prox,
  'https': prox,
}
# html = get_html(START_URL, proxies=proxies)
try:
    html = get_html('https://www.chita.ru/', proxies=proxies, timeout=10)
except Exception as e:
    print(e)


Wall time: 4.19 s


In [500]:
soup = BeautifulSoup(html.text, 'html.parser')
search_table = soup.find_all('table', class_='table table-hover table-filled search-results')
search_tr = 

links = search_table[0].tbody.find_all('a')
dates = search_table[0].tbody.find_all('td', class_='nowrap')
links[5]

<a class="visited" href="/firms/ooo-velesstroi/35328/" target="_blank">ООО "Велесстрой"</a>

In [7]:
def write_log(text, filename='log.txt'):
    curtime = datetime.now().isoformat().split('.')[0].replace('T', ' ')
    try:
        f = open(filename, 'a')
        f.write(curtime + ' ' + text + '\n')
    finally:
        f.close()

### Функция возвращает результат HTML запроса

In [8]:
def get_html(url, proxy=None, timeout=None):
    if proxy:
        proxies = {
          'http': proxy[0],
          'https': proxy[0],
        }
        proxy_id = proxy[1]
        proxy_log = proxy[0]
    else:
        proxies = None
        proxy_log = 'NO PROXY'
        proxy_id = 0
    
    try:
        r = requests.get(url, headers=HEADERS, proxies=proxies, timeout=timeout)
        return r
    except Exception as e:
#         write_log(proxy[0] + ' ' + str(e))
        write_log(proxy[0] + ' Connection error')
        mysql_write('UPDATE proxy_servers SET fall_count = fall_count + 1 WHERE id =' + str(proxy_id))
        return False

### Функция возвращает элемент url адреса from для последней страницы

In [9]:
def get_last_page(html):
    soup = BeautifulSoup(html.text, 'html.parser')
    search_pagination = soup.find_all('li', class_='pagi-item')
    return int(search_pagination[-1].a.get('href').split('from=')[1])

### Выдает свободный proxy из базы и отмечает его как использованный

In [10]:
def give_me_proxy():
    # Возвращает список из адреса прокси и id в базе, либо 0, если подходящий прокси не нашелся
    min_time = 120 # Таймаут в секундах, после которого прокси можно использовать снова
    fail_count = 10
    timeout = round(time.time()) - min_time
    query = f"""
    SELECT ps.id as id,
           CONCAT(pt.type, "://", ps.ip, ":", ps.port) as ip
           FROM proxy_servers ps
    LEFT JOIN used_ip ui ON ps.id = ui.proxy_server_id
    INNER JOIN proxy_type pt ON ps.proxy_type_id = pt.id
    WHERE (ui.project_id = {PROJECT_ID} AND ui.timestamp < {timeout} OR ui.timestamp IS NULL) AND ps.fall_count < {fail_count}
    ORDER BY ps.fall_count, ui.timestamp
    LIMIT 1;
    """
    cursor = mysql_read(query)
    if cursor.rowcount == 0: # Если не нашелся "свободный" прокси
        return 0
    else:
        result = cursor.fetchone()
        ip = result['ip']
        ip_id = result['id']
        if mysql_read(f'SELECT 1 FROM used_ip WHERE project_id = {PROJECT_ID} AND proxy_server_id = {ip_id}').rowcount == 0:
            query = \
            f'INSERT INTO used_ip (proxy_server_id, project_id, timestamp) VALUES ({ip_id}, {PROJECT_ID}, {timeout + min_time})'
        else:
            query = \
            f'UPDATE used_ip SET timestamp = {timeout + min_time} WHERE proxy_server_id = {ip_id} AND project_id = {PROJECT_ID}'
#         print(query)
        mysql_write(query)
        return [ip, ip_id]

In [870]:
%%time
ip = '\
131.255.239.38:3128'
prox = 'https://' + ip
# prox = 'socks4://' + ip
# prox = 'socks5://' + ip


#df_init()
# html = get_html(START_URL, proxies=proxies)

start_page = 124900
last_page = 130000

for i in range(start_page, last_page, REC_PER_PAGE):
    curl = START_URL + '&from=' + str(i)
    html = False
    while not html:
        proxy = 0
        while not proxy:
            proxy = give_me_proxy()
            if not proxy:
                write_log('No avaliable proxy')
                sleep(10)
        html = get_html(curl, proxy, 8)
        if html and BeautifulSoup(html.text, 'html.parser').html.\
                                    text.find('Превышен максимальный лимит') > -1: # Если забанили ip
            html = False
            timeout = round(time.time()) + 86460 # Не используем этот прокси следующие сутки + 1 мин.
            query = \
            f'UPDATE used_ip SET timestamp = {timeout} WHERE proxy_server_id = {proxy[1]} AND project_id = {PROJECT_ID}'
            write_log(proxy[0] + ' BANNED ON THE SERVER')
            mysql_write(query)
    parse_list_page(html)
    mysql_write('UPDATE proxy_servers SET fall_count = 0 WHERE id =' + str(proxy[1]))
    progress(i - start_page + REC_PER_PAGE, last_page - start_page, suffix= str(i) + ' ')
#     sleep(randint(60, 120))

Wall time: 50min 39s=====================] 100.0% ...129980 


In [895]:
# df.info()
#df['Sponsor'].value_counts()
count = str(len(df['Id'].values)) + '_' + datetime.now().isoformat().split('.')[0].replace(':', '-').replace('T', '_')
# df.to_excel(f'first_{count}.xlsx')
df[['Id', 'Type', 'Name', 'Sponsor', 'Start_date', 'End_date']].to_excel(f'first_{count}.xlsx')
#df.to_pickle(f'first_{count}.pkl')
#df.to_csv(f'first_{count}.csv', encoding='utf-8-sig')

## TEST ZONE 

In [11]:
def give_me_proxy_mt(nothing=''):
    # Возвращает список из адреса прокси и id в базе, либо 0, если подходящий прокси не нашелся
    min_time = 120 # Таймаут в секундах, после которого прокси можно использовать снова
    fail_count = 10
    timeout = round(time.time()) - min_time
    query = f"""
    SELECT ps.id as id,
           CONCAT(pt.type, "://", ps.ip, ":", ps.port) as ip
           FROM proxy_servers ps
    LEFT JOIN used_ip ui ON ps.id = ui.proxy_server_id
    INNER JOIN proxy_type pt ON ps.proxy_type_id = pt.id
    WHERE (ui.project_id = {PROJECT_ID} AND ui.timestamp < {timeout} OR ui.timestamp IS NULL) AND ps.fall_count < {fail_count}
    ORDER BY ps.fall_count, ui.timestamp
    LIMIT 1;
    """
    lock.acquire()
    cursor = mysql_read(query)
    if cursor.rowcount == 0: # Если не нашелся "свободный" прокси
        lock.release()
        return 0
    else:
        result = cursor.fetchone()
        ip = result['ip']
        ip_id = result['id']
        if mysql_read(f'SELECT 1 FROM used_ip WHERE project_id = {PROJECT_ID} AND proxy_server_id = {ip_id}').rowcount == 0:
            query = \
            f'INSERT INTO used_ip (proxy_server_id, project_id, timestamp) VALUES ({ip_id}, {PROJECT_ID}, {timeout + min_time})'
        else:
            query = \
            f'UPDATE used_ip SET timestamp = {timeout + min_time} WHERE proxy_server_id = {ip_id} AND project_id = {PROJECT_ID}'
#         print(query)
        mysql_write(query)
        lock.release()
        return [ip, ip_id]

In [75]:
# Возвращает список URL для этапа 0.0 (адреса из которых будут взят список ссылок для 0-го этапа)

def get_url_step_0_0():
    result = []
    u0 = 'https://www.b2b-center.ru/market/?searching=1&company_type=2&price_currency=0&date=1'
    u2 = '&trade=all&show=archive&from=0'
    
    result.append(u0 + '&date_start_dmy=01.10.2002&date_end_dmy=31.12.2007' + u2)
    result.append(u0 + '&date_start_dmy=01.01.2008&date_end_dmy=31.12.2011' + u2)
    for i in range(2012, 2021):
        if 2012 <= i <= 2017:
            result.append(u0 + f'&date_start_dmy=01.01.{i}&date_end_dmy=30.06.{i}' + u2)
            result.append(u0 + f'&date_start_dmy=01.07.{i}&date_end_dmy=31.12.{i}' + u2)
        else:
            result.append(u0 + f'&date_start_dmy=01.01.{i}&date_end_dmy=30.04.{i}' + u2)
            result.append(u0 + f'&date_start_dmy=01.05.{i}&date_end_dmy=31.08.{i}' + u2)
            if i == 2020:
                result.append(u0 + f'&date_start_dmy=01.09.{i}&date_end_dmy=30.09.{i}' + u2)
            else:
                result.append(u0 + f'&date_start_dmy=01.09.{i}&date_end_dmy=31.12.{i}' + u2)
            
    return(result)

urls00 = get_url_step_0_0()
print(len(urls00))
print(urls00[-1])

23
https://www.b2b-center.ru/market/?searching=1&company_type=2&price_currency=0&date=1&date_start_dmy=01.09.2020&date_end_dmy=30.09.2020&trade=all&show=archive&from=0


In [76]:
%%time
# Добываем ссылки для 0-го этапа (ссылки на страницы по 20 заголовков конкурсов на каждой)
step_0_urls = []
for link in urls00:
    html = 0
    while not html:
        proxy = 0
        while not proxy:
            proxy = give_me_proxy()
            html = get_html(link, proxy, 8)
            if html and BeautifulSoup(html.text, 'html.parser').html.\
                                        text.find('Превышен максимальный лимит') > -1: # Если забанили ip
                html = False
                timeout = round(time.time()) + 86460 # Не используем этот прокси следующие сутки + 1 мин.
                query = \
                f'UPDATE used_ip SET timestamp = {timeout} WHERE proxy_server_id = {proxy[1]} AND project_id = {PROJECT_ID}'
                write_log(proxy[0] + ' BANNED ON THE SERVER')
                mysql_write(query)
    
    last_page = get_last_page(html)
    for i in range(0, last_page + REC_PER_PAGE, 20):
        step_0_urls.append(link.replace('&from=0', '&from=' + str(i)))
    print('.', end=' ')
    
with open('step_0_links.txt', 'w') as file:
    for line in step_0_urls:
        file.write(line + '\n')

. . . . . . . . . . . . . . . . . . . . . . . Wall time: 6min 27s


In [134]:
%%time
while False:
    # Добавляем ссылки порциями по 10000
    rec_count = len(step_0_urls)
    recs_per_insert = 10000
    steps_count = rec_count // recs_per_insert
    for i in range(steps_count):
        insetr_values = ',\n'.join(list(map(lambda x: '("' + x + '")', step_0_urls[i*recs_per_insert:(i+1)*recs_per_insert])))
        insert_query = 'INSERT INTO step0_urls (url) VALUES ' + insetr_values
        mysql_write2(insert_query)

    insetr_values = ',\n'.join(list(map(lambda x: '("' + x + '")', step_0_urls[(i+1)*recs_per_insert:])))
    insert_query = 'INSERT INTO step0_urls (url) VALUES ' + insetr_values
    mysql_write2(insert_query)

Wall time: 0 ns


In [12]:
def get_html_mt(url, proxy=None, timeout=None):
    if proxy:
        proxies = {
          'http': proxy[0],
          'https': proxy[0],
        }
        proxy_id = proxy[1]
        proxy_log = proxy[0]
    else:
        proxies = None
        proxy_log = 'NO PROXY'
        proxy_id = 0
    
    try:
        r = requests.get(url, headers=HEADERS, proxies=proxies, timeout=timeout)
        return r
    except Exception as e:
        with log_lock:
#             write_log(proxy[0] + ' ' + str(e))
            write_log(proxy[0] + ' Connection error')
        with lock:
            mysql_write('UPDATE proxy_servers SET fall_count = fall_count + 1 WHERE id =' + str(proxy_id))
        return False

In [13]:
def ban_proxy_mt(id):
    timeout = round(time.time()) + 86460 # Не используем этот прокси следующие сутки + 1 мин.
    query = \
    f'UPDATE used_ip SET timestamp = {timeout} WHERE proxy_server_id = {id} AND project_id = {PROJECT_ID}'
    with log_lock:
        write_log(proxy[0] + ' BANNED ON THE SERVER')
    with lock:
        mysql_write(query)

### <font color="red">ПАРСИНГ MT</font>

In [14]:
def parse_list_page_mt(html):
    add_list = []
    search_table = html
    links = search_table[0].tbody.find_all('a')
    dates = search_table[0].tbody.find_all('td', class_='nowrap')

    for i in range(0, len(links), 2):
        auc_id = links[i].text.split('\n')[0].split('№')[1].strip() # Номер торгов
        auc_type = links[i].text.split('\n')[0].split('№')[0].strip() # Вид торгов
        auc_name = links[i].text.split('\n')[1].strip() # if len(links[i].text.split('\n')) > 1 else '11' # Название торгов
        auc_name = np.nan if len(auc_name) == 0 else auc_name
        sponsor = np.nan if len(links[i+1].text.strip()) == 0 else links[i+1].text.strip() # Наименование органзатора
        auc_link = links[i].get('href').split('#')[0] # Ссылка на страницу торгов
        start_date = dates[i].text # Колонка "Опубликовано"
        end_date = dates[i+1].text # Колонка "Актуально до"
        add_list.append([auc_id, auc_type, auc_name, sponsor, auc_link, start_date, end_date])
    return add_list

In [15]:
def parse_list_page_file_mt(html):
    add_list = []
    soup = BeautifulSoup(html, 'html.parser')
    search_table = soup.find_all('table', class_='table table-hover table-filled search-results')
    links = search_table[0].tbody.find_all('a')
    dates = search_table[0].tbody.find_all('td', class_='nowrap')

    for i in range(0, len(links), 2):
        auc_id = links[i].text.split('\n')[0].split('№')[1].strip() # Номер торгов
        auc_type = links[i].text.split('\n')[0].split('№')[0].strip() # Вид торгов
        auc_name = links[i].text.split('\n')[1].strip() # if len(links[i].text.split('\n')) > 1 else '11' # Название торгов
        auc_name = np.nan if len(auc_name) == 0 else auc_name
        sponsor = np.nan if len(links[i+1].text.strip()) == 0 else links[i+1].text.strip() # Наименование органзатора
        auc_link = links[i].get('href').split('#')[0] # Ссылка на страницу торгов
        start_date = dates[i].text # Колонка "Опубликовано"
        end_date = dates[i+1].text # Колонка "Актуально до"
        add_list.append([auc_id, auc_type, auc_name, sponsor, auc_link, start_date, end_date])
    return add_list

In [16]:
def give_me_html_mt(url):
    url_id = url[0]
    curl = url[1]

    while True:
        proxy = give_me_proxy_mt()
        if not proxy: # Если нет свободных прокси
            with log_lock:
                write_log('No avaliable proxy')
            sleep(5)
            continue
        html = get_html_mt(curl, proxy, 8)
        # Если html не вернулся или вернулся не 200-й код, начинаем заново
        if not html or html.status_code != 200:
            continue
        # Если забанили ip на сервере, начинаем заново
        if BeautifulSoup(html.text, 'html.parser').html.text.find('Превышен максимальный лимит') > -1:
            continue
        # Тут мы получили нормальный HTML не забаненный на сервере
        with lock:
            mysql_write('UPDATE proxy_servers SET fall_count = 0 WHERE id =' + str(proxy[1]))
        soup = BeautifulSoup(html.text, 'html.parser')
        search_table = soup.find_all('table', class_='table table-hover table-filled search-results')
        with open('html/step01/' + str(url_id).zfill(7) +'.html', 'w', encoding='utf-8') as file:
            file.write(str(search_table))
        return [search_table, url_id]

In [17]:
def sql_date(strdate):
#   Форматирует дату под SQL формат
    cur_date = strdate.split(' ')[0].split('.')
    cur_time = strdate.split(' ')[1]
    sql_date = cur_date[2] + '-' + cur_date[1] + '-' + cur_date[0] + ' ' + cur_time
    return sql_date

def insert_step1_rec(values):
    for item in values:
        #Проверяем есть ли в базе тендер с таким номером
        query = 'SELECT 1 FROM tender WHERE  tender_id = %s'
        if mysql_read2(query, item[0]).rowcount > 0:
            query = 'SELECT 1 FROM tender WHERE  tender_id = %s AND name = %s AND start_date = %s'
            if mysql_read2(query, (item[0], item[2], sql_date(item[5]))).rowcount > 0:
                write_log(f'дубль тендера step1, id={item[0]} {HOST + item[4]}', 'important.log')
                continue
            
        # Проверяем есть ли такой тип тендера и если нет - добавляем
        query = f'SELECT id FROM tender_type WHERE name = %s'
        result = mysql_read2(query, item[1])
        if result.rowcount > 0:
            type_id = result.fetchone()[0]
        else:
            query = f'INSERT INTO tender_type (name) VALUES (%s)'
            type_id = mysql_write2(query, item[1]).lastrowid
            
        # Проверяем есть ли такой заказчик и если нет - добавляем
        query = f'SELECT id FROM sponsor WHERE name = %s'
        result = mysql_read2(query, item[3])
        if result.rowcount > 0:
            sponsor_id = result.fetchone()[0]
        else:
            query = f'INSERT INTO sponsor (name) VALUES (%s)'
            sponsor_id = mysql_write2(query, item[3]).lastrowid
        
        query = 'INSERT INTO tender(tender_id, tender_type_id, name, sponsor_id, start_date, end_date, link) '
        query += 'VALUES (%s, %s, %s, %s, %s, %s, %s)'
        mysql_write2(query, (item[0], type_id, item[2], sponsor_id, sql_date(item[5]), sql_date(item[6]), item[4]))

In [323]:
%%time
iters = 10
for i in range(iters):
    # Получаем порцию ссылок для парсинга
    rec_per_time = 100
    query = f'SELECT id, url FROM step0_urls WHERE status = 0 ORDER BY id LIMIT {rec_per_time}'
    s1_links = list(mysql_read2(query).fetchall())
    
    if len(s1_links) == 0:
        break
    
    # Собираем id url и обновляем их на статус 1 (в обработке)
    link_ids = []
    for item in s1_links:
        link_ids.append(str(item[0]))
    link_ids = '(' + ', '.join(link_ids) + ')'
    mysql_write2('UPDATE step0_urls SET status = 1 WHERE id IN ' + link_ids)

    # Инициализируем блокировки
    lock = threading.Lock()
    log_lock = threading.Lock()

    progress(i, iters, suffix= 'Скачиваем ')
    
    # Запускаем многопоточное скачивание
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        htmls = executor.map(give_me_html_mt, s1_links)

    progress(i, iters, suffix= 'Парсим ')
    
    # Парсим полученный результат
    htmls = list(htmls)
    values = []
    for item in htmls:
        values.extend(parse_list_page_mt(item[0]))

    progress(i, iters, suffix= 'Добавляем ')

    # Добавляем базу то, что напарсили
    insert_step1_rec(values)

    # Устанавливаем статус 2 (завершено) для обработанных ссылок
    mysql_write2('UPDATE step0_urls SET status = 2 WHERE id IN ' + link_ids);

Wall time: 2min 9s-----------------------] 0.0% ...Добавляем 


In [ ]:
start_file = 1;
end_file = 2300
for i in range(start_file, end_file + 1):
    with open('html/step01/' + str(i).zfill(7) +'.html', 'r', encoding='utf-8') as file:
        html = file.read()
    values = parse_list_page_file_mt(html)
    insert_step1_rec(values)
    progress(i - start_file + 1, end_file - start_file)

In [350]:
i

210